In [1]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests

In [2]:
# Load the CLIP model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "openai/clip-vit-base-patch16"
processor = CLIPProcessor.from_pretrained(model_name)
model = CLIPModel.from_pretrained(model_name).to(device)

E:\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\darkn\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
# Function to find the object in an image
def find_object_in_image(object_name, image_path):
    # Encode the object name and image
    text = object_name
    image = Image.open(requests.get(image_path, stream=True).raw)
    inputs = processor(text, images=image, return_tensors="pt").to(device)

    # Get the image and text embeddings
    with torch.no_grad():
        image_features = model.get_image_features(inputs.pixel_values).to(device)
        text_features = model.get_text_features(inputs.input_ids).to(device)

    # Compute the similarity score between image and text features
    similarity = (image_features @ text_features.T).squeeze(0)
    return similarity.item()

In [5]:
# Example usage
object_name = "chair"
image_url = "https://img.homejournal.com/202005/5eb28b30b7c7c.jpg"
similarity_score = find_object_in_image(object_name, image_url)
print(f"Similarity score for '{object_name}' in the image: {similarity_score:.2f}")

Similarity score for 'chair' in the image: 23.61
